In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import math
import numpy as np
import collections
from numpy import linalg as la
import time
import json

# Part-1) Tweets pre-processings

In [3]:
"""
This dataset consists only of one line where all the 2399 tweets by the WHO twitter account. 
We use the json python library to load this text as a dictionary since the dataset is stored 
in JSON format. 
"""
docs_path = 'dataset_tweets_WHO.txt'
with open(docs_path) as fp:
    lines = fp.readline()
tweets = json.loads(lines)

In [4]:
print("First tweet structure:")
tweets['0']

First tweet structure:


{'created_at': 'Wed Oct 13 09:15:58 +0000 2021',
 'id': 1448215930178310144,
 'id_str': '1448215930178310144',
 'full_text': "It's International Day for Disaster Risk Reduction\n\n#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.\n\nStart learning today &amp; be #Ready4Response:\n👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS",
 'truncated': False,
 'display_text_range': [0, 274],
 'entities': {'hashtags': [{'text': 'OpenWHO', 'indices': [52, 60]},
   {'text': 'Ready4Response', 'indices': [232, 247]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/hBFFOF0xKL',
    'expanded_url': 'https://bit.ly/3wCa0Dr',
    'display_url': 'bit.ly/3wCa0Dr',
    'indices': [251, 274]}],
  'media': [{'id': 1448215398814560259,
    'id_str': '1448215398814560259',
    'indices': [275, 298],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1448215398814560259/pu/img/0CO

In [5]:
tweets['0'].keys()

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])

In [6]:
print("First three full text tweets:\n")
for i in range(3):
    print("Tweet number",i+1,":\n")
    print(tweets[str(i)]["full_text"])
    print("\n")
    

First three full text tweets:

Tweet number 1 :

It's International Day for Disaster Risk Reduction

#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.

Start learning today &amp; be #Ready4Response:
👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS


Tweet number 2 :

#COVID19 has shown how health emergencies and disasters affect entire communities – especially those with weak health systems, and vulnerable populations like migrants, indigenous peoples, and those living in fragile humanitarian conditions. https://t.co/jpUQpnu0V1


Tweet number 3 :

It's International Day for Disaster Risk Reduction
 
To better respond to emergencies countries must:
✅ invest in health care systems
✅ achieve gender equity
✅ protect marginalised groups
✅ ensure ready &amp; equitable access to supplies
 
A strong &amp; resilient health system is 🔑 https://t.co/5NALyjIymp




In [7]:
def process_tweet(line):
    """
    Pre-process the tweet text removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be pre-processed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line = line.lower() ## Transform in lowercase
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    line = tokenizer.tokenize(line)
    line = [word for word in line if word not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line = [stemmer.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    i = 0
    for word in line:
        if word[0:4] == 'http':
            line = line[:i]
        i+=1
    ## END CODE
    return line

In [8]:
"""
Here for every tweet we extract the 'full_text' variable since we will only 
search inside the text of each tweet. Thus, we pre-process each tweet and 
store it in a new dictionary called proc_tweets for later use
"""
proc_tweets = {}
for tweet_id, tweet in zip(tweets.keys(),tweets.values()):
    proc_tweets[int(tweet_id)] = process_tweet(tweet['full_text'])

In [9]:
print("First five processed tweets\n")
for i in range(3):
    print(proc_tweets[i])
    print("\n")
    

First five processed tweets

['intern', 'day', 'disast', 'risk', 'reduct', 'openwho', 'launch', 'multi', 'tier', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'respons', 'start', 'learn', 'today', 'amp', 'ready4respons']


['covid19', 'shown', 'health', 'emerg', 'disast', 'affect', 'entir', 'commun', 'especi', 'weak', 'health', 'system', 'vulner', 'popul', 'like', 'migrant', 'indigen', 'peopl', 'live', 'fragil', 'humanitarian', 'condit']


['intern', 'day', 'disast', 'risk', 'reduct', 'better', 'respond', 'emerg', 'countri', 'must', 'invest', 'health', 'care', 'system', 'achiev', 'gender', 'equiti', 'protect', 'marginalis', 'group', 'ensur', 'readi', 'amp', 'equit', 'access', 'suppli', 'strong', 'amp', 'resili', 'health', 'system']




Lab 2:

INDEXING:
1. Create inverted index
2. Make a proposal of 5 queries that will be used to evaluate our search engine
3. Apply a TF-IDF ranking to your results

EVALUATION:
1. Set the ground truth for each document and queryy (binary way)
2. Evaluate the algorithm with:
    2.1 Precision @K
    2.2 Average Precision @K 
    2.3 Mean Average Precision
    2.4 Mean Reciprocal Rank
    2.5 Normalizewd Discounted Cummulative Gain
3. Choose one vector rpresentation (TF-IDF or word2vec)

In [10]:
def get_tweet_info(tweet):
    Tweet = tweet['full_text']
    Username = tweet['user']['name']
    Date = tweet['created_at']
    #Hashtags = tweet['entities']['hashtags']['text']

    Hashtags = []
    hashtags_list = tweet['entities']['hashtags']
    for hashtag in hashtags_list:
        Hashtags.append(hashtag['text'])

    Likes = tweet['favorite_count']
    Retweets = tweet['retweet_count']
    Url = f"https://twitter.com/{tweet['user']['screen_name']}/status/{tweet['id_str']}"
    info = [Tweet, Username, Date, Hashtags, Likes, Retweets, Url]
    return info

In [19]:
def create_index(tweets):
    """
    Generates the index from our database to perform queries from
    
    Argument:
    tweets -- collection of tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = {}
    id_index = {}
    tf = defaultdict(list)
    df = defaultdict(int)
    idf = defaultdict(float)
    numDocuments = len(tweets)
    for i in range(numDocuments):
        tweet = tweets[str(i)]
        terms = process_tweet(tweet['full_text']) #get tweet text
        info = get_tweet_info(tweet) # get "document" info
        id_tweet = tweet['id']
        id_index[id_tweet]=info
        
        
        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                index[term].append(id_tweet)  
                
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                index[term]= [id_tweet]#'I' indicates unsigned int (int in Python)
            
        norm=0
        for term,ids in index.items():
            norm += len(ids)**2
        norm = math.sqrt(norm)
        
        for term,ids in index.items():
            tf[term].append(np.round(len(ids)/norm,4))
            df[term] += 1
        
        for term in index:
            idf[term] = np.round(np.log(float(numDocuments/df[term])),4)
        
    return index, tf,df,idf,id_index


In [20]:
start_time = time.time()
index, tf,df,idf,id_index = create_index(tweets)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 122.6 seconds


In [21]:
def search(query, index):
    query = process_tweet(query)
    first = True
    docs = {}
    for term in query:
        try:
            list_docs = index[term]
            if first:
                docs = set(list_docs)
                first = False
            else:
                docs &= set(list_docs)
            
        except:
            break
    docs=list(docs)
    return docs           

In [22]:
# def search(query, index):
#     query = process_tweet(query)
    

In [47]:
print("Insert your query:\n")
query = input()
docs = search(query,index)

print("Top 10 results:\n")
for d_id in docs[:10]:
    print(f"{id_index[d_id]}\n")

Insert your query:

top
Top 10 results:

["RT @DrTedros: I thanked the Prime Minister for 🇰🇼's support to #COVAX, but also to various @WHO emergency responses. 🇰🇼 is one of the top 1…", 'World Health Organization (WHO)', 'Wed Jul 28 19:07:57 +0000 2021', ['COVAX'], 0, 37, 'https://twitter.com/WHO/status/1420461039926358017']

['Dr @Kate_L_OBrien explains what we know so far about  #COVID19 vaccine "booster" shots, and why making sure at-risk people 𝙚𝙫𝙚𝙧𝙮𝙬𝙝𝙚𝙧𝙚 get 1st and 2nd doses should be the top priority. ⬇️\n\n#VaccinEquity https://t.co/igI76vF5tK', 'World Health Organization (WHO)', 'Thu Aug 05 20:40:05 +0000 2021', ['COVID19', 'VaccinEquity'], 574, 279, 'https://twitter.com/WHO/status/1423383330393935873']

['@DrTedros "What happened to you should never happen to anyone. It’s inexcusable. It is my top priority to ensure that the perpetrators are not excused, but are held to account"-@DrTedros https://t.co/CbCTLWbiHi', 'World Health Organization (WHO)', 'Tue Sep 28 13:41:01 +0000 

In [53]:
def rank_documents(terms, docs, index, idf, tf, title_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    #HINT: use when computing tf for query_vector

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex]=query_terms_count[term]/query_norm * idf[term] 

        # Generate doc_vectors for matching docs
        for doc in index[term]:
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc] * idf[term]  # TODO: check if multiply for idf

    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine similarity
    # see np.dot
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)
    #print ('\n'.join(result_docs), '\n')
    return result_docs

In [57]:
def search_tf_idf(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    
    query = process_tweet(query)
    docs = set()
    for term in query:
        try:
            list_docs = index[term]
            if first:
                docs = set(list_docs)
                first = False
            else:
                docs &= set(list_docs)
            
        except:
            break
            
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf, id_index)
    
    return ranked_docs

In [ ]:
print("Insert your query (i.e.: Computer Science):\n")
query = input()
ranked_docs = search_tf_idf(query, index)
top = 10

print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))

Insert your query (i.e.: Computer Science):

covid
No results found, try again
covid
No results found, try again
